evaluate patterns if full embeddings are already generated

In [1]:
import numpy as np
import os 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet
from sklearn.metrics import auc, roc_curve, roc_auc_score, balanced_accuracy_score, \
                            mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_predict, train_test_split, cross_validate, \
                                    LeaveOneGroupOut, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

In [2]:
list_auc_values = []

# Classification

FIP right

In [ ]:
# parameters
## Champollion
embds_dir = '/neurospin/dico/jlaval/Output/V1_FINAL_06-2025/FIP_right/name06-17-01_29/FIP_right_train_test_embeddings/full_embeddings.csv'
## B-VAE
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/FIP_right/32dims/FIP_right_custom_embeddings/unformated_embeddings.csv' # 32 dims is best
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/FIP_right/75dims/FIP_right_custom_embeddings/unformated_embeddings.csv'
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/FIP_right/256dims/FIP_right_custom_embeddings/unformated_embeddings.csv'
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/FIP_labels.csv'
label = 'Right_FIP'
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/Right/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/Right/test_split.csv'
label_type = 'binary'
subject_name = 'Subject'

SOr_left

In [ ]:
# parameters
## Champollion
#embds_dir = '/neurospin/dico/jlaval/Output/V1_FINAL_06-2025/SOr_left/name14-12-56_162/hcp_random_embeddings/full_embeddings.csv'
## B-VAE
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/SOr_left/32dims/unformated_embeddings.csv'
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/SOr_left/75dims/unformated_embeddings.csv'
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/SOr_left/256dims/unformated_embeddings.csv'
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_OFC_labels.csv'
label = 'Left_OFC'
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/orbital_patterns/Troiani/Left/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/orbital_patterns/Troiani/Left/test_split.csv'
label_type = 'multiclass'
subject_name = 'Subject'

LARGE_CINGULATE_right

In [144]:
# parameters
## Champollion
#embds_dir = '/neurospin/dico/jlaval/Output/V1_FINAL_06-2025/LARGE_CINGULATE_right/name07-22-35_179/Right_PCS_random_embeddings/full_embeddings.csv'
## B-VAE
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/LARGE_CINGULATE_right/32dims/unformated_embeddings.csv' # 32 dims is best
#embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/LARGE_CINGULATE_right/75dims/unformated_embeddings.csv'
embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/LARGE_CINGULATE_right/256dims/unformated_embeddings.csv'
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/subjects_labels.csv'
label = 'Right_PCS'
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/splits/Right/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/splits/Right/test_split.csv'
label_type = 'binary'
subject_name = 'long_name'

In [145]:
# load
embds = pd.read_csv(embds_dir)
embds.columns = ['ID'] + [f'dim{i}' for i in range(embds.shape[1]-1)]
# remove duplicates
embds = embds.drop_duplicates(subset=['ID'])
labels = pd.read_csv(labels_dir)

## LINEAR METHODS ON IMAGES

Large Cingulate

In [ ]:
embds_dir = 'linear_eval_cingulate' ## just for saving results
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/subjects_labels.csv'
label = 'Right_PCS'
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/splits/Right/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/splits/Right/test_split.csv'
label_type = 'binary'
subject_name = 'long_name'

# load the images
skeletons = np.load('/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/crops/2mm/LARGE_CINGULATE./mask/Rskeleton.npy')
skel_names = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/crops/2mm/LARGE_CINGULATE./mask/Rskeleton_subject.csv')
# flatten the skeletons
skeletons = skeletons.reshape(skeletons.shape[0], -1)
# binarize the skeletons
skeletons[skeletons > 0] = 1
# create a DataFrame
embds = pd.DataFrame(skeletons, columns=[f'dim{i}' for i in range(skeletons.shape[1])])
# add ID column
embds['ID'] = skel_names['Subject']
# remove duplicates
embds = embds.drop_duplicates(subset=['ID'])

labels = pd.read_csv(labels_dir)

SOr

In [232]:
embds_dir = 'linear_eval_SOr' ## just for saving results
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_OFC_labels.csv'
label = 'Left_OFC'
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/orbital_patterns/Troiani/Left/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/orbital_patterns/Troiani/Left/test_split.csv'
label_type = 'multiclass'
subject_name = 'Subject'

# load the images
skeletons = np.load('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.Or./mask/Lskeleton.npy')
skel_names = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.Or./mask/Lskeleton_subject.csv')
# flatten the skeletons
skeletons = skeletons.reshape(skeletons.shape[0], -1)
# binarize the skeletons
skeletons[skeletons > 0] = 1
# create a DataFrame
embds = pd.DataFrame(skeletons, columns=[f'dim{i}' for i in range(skeletons.shape[1])])
# add ID column
embds['ID'] = skel_names['Subject']
# remove duplicates
embds = embds.drop_duplicates(subset=['ID'])

labels = pd.read_csv(labels_dir)

FIP

In [3]:
embds_dir = 'linear_eval_FIP' ## just for saving results
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/FIP_labels.csv'
label = 'Right_FIP'
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/Right/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/Right/test_split.csv'
label_type = 'binary'
subject_name = 'Subject'

# load the images
skeletons = np.load('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P./mask/Rskeleton.npy')
skel_names = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P./mask/Rskeleton_subject.csv')
# flatten the skeletons
skeletons = skeletons.reshape(skeletons.shape[0], -1)
# binarize the skeletons
skeletons[skeletons > 0] = 1
# create a DataFrame
embds = pd.DataFrame(skeletons, columns=[f'dim{i}' for i in range(skeletons.shape[1])])
# add ID column
embds['ID'] = skel_names['Subject']
# remove duplicates
embds = embds.drop_duplicates(subset=['ID'])

labels = pd.read_csv(labels_dir)

In [4]:
# restrict embds to Subjects with labels
embds = embds[embds['ID'].isin(labels[subject_name])].reset_index(drop=True)
# same for labels
labels = labels[labels[subject_name].isin(embds['ID'])].reset_index(drop=True)
# align labels and embds on 'ID'
labels = labels.merge(embds[['ID']], left_on=subject_name, right_on='ID', how='right')
# order all by ID
embds = embds.sort_values(by='ID').reset_index(drop=True)
labels = labels.sort_values(by='ID').reset_index(drop=True)
subjects = embds['ID']

# define X, Y and subjects
X = embds.drop(columns=['ID'])
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = labels[label]

print(X.shape, Y.shape)


subs_embeddings = pd.DataFrame({'ID': subjects, 'X': list(X), 'Y': Y})

(390, 77220) (390,)


In [5]:
root_dir = '/'.join(splits_basedir.split('/')[:-1])
basedir = splits_basedir.split('/')[-1]
splits_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir) if f.startswith(basedir) and '.csv' in f]
splits_subs = [pd.read_csv(file, header=None) for file in splits_dirs]
labels = np.concatenate([[i] * len(K) for i, K in enumerate(splits_subs)])
splits_subs_and_labels = pd.concat(splits_subs)
splits_subs_and_labels.columns=['ID']
splits_subs_and_labels['labels'] = labels
df = subs_embeddings.merge(splits_subs_and_labels, on='ID')
groups, X_train_val, Y_train_val = df['labels'], np.vstack(df['X'].values), df['Y']
logo = LeaveOneGroupOut()
cv = [*(logo.split(X_train_val, Y_train_val, groups=groups))]

In [6]:
## LINEAR PROBING
mode = 'linear'
parameters={'l1_ratio': np.linspace(0,1,11), 'C': [10**k for k in range(-3,4)]}
model = LogisticRegression(solver='saga', penalty='elasticnet',
                            max_iter=2000, random_state=0)
clf = GridSearchCV(model, parameters, cv=cv, scoring='roc_auc_ovr_weighted', refit=True, n_jobs=-1)

In [ ]:
## KNN
# mode = 'knn'
# parameters = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15], 'leaf_size': [1, 5, 10, 20, 30], 'weights': ['uniform', 'distance'], 'metric': ['minkowski', 'manhattan', 'cosine']}
# model = KNeighborsClassifier(n_jobs=-1)
# clf = GridSearchCV(model, parameters, cv=cv, scoring='roc_auc_ovr_weighted', refit=True, n_jobs=-1)

In [7]:
# fit cross-validation
clf.fit(X_train_val,Y_train_val)
print(f'best params : {clf.best_params_}')
print(f'best score : {clf.best_score_}')
md = clf.best_estimator_
labels_proba = cross_val_predict(md, X_train_val, Y_train_val, cv=cv, method='predict_proba') # why does it give slightly diff results ??
if label_type == 'binary':
    cross_val_roc_auc = roc_auc_score(Y_train_val, labels_proba[:, 1])
elif label_type == 'multiclass':
    cross_val_roc_auc = roc_auc_score(Y_train_val, labels_proba, multi_class='ovr', average='weighted')
print(f'Cross-val ROC AUC: {cross_val_roc_auc:.3f}')

/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-pack

best params : {'C': 0.001, 'l1_ratio': np.float64(0.0)}
best score : 0.5976788694781351
Cross-val ROC AUC: 0.588


In [8]:
# use parameters for test
test_subjects = pd.read_csv(test_subs_dir, header=None)
test_subjects.columns = ['ID']
subs_embeddings_test = subs_embeddings.merge(test_subjects, on='ID')
# check that there is no intersection between train val and test subjects
print(f"Check intersection between train/val and test : {set(splits_subs_and_labels['ID']).intersection(set(subs_embeddings_test['ID']))}")
X_test, Y_test = np.vstack(subs_embeddings_test['X'].to_numpy()), subs_embeddings_test['Y']
# take best parameters obtained in cross-validation and fit on train+val ## NO NEED TO DO THIS, ALREADY DONE WITH REFIT=TRUE
# if mode == 'linear':
#     print('Using linear probing')
#     md = LogisticRegression(solver='saga', penalty='elasticnet',
#                                 max_iter=2000, random_state=0, **clf.best_params_)
# elif mode == 'knn':
#     print('Using KNN')
#     md = KNeighborsClassifier(n_jobs=-1, **clf.best_params_)
# md.fit(X_train_val,Y_train_val)
labels_proba_test = md.predict_proba(X_test)
if label_type == 'binary':
    test_roc_auc = roc_auc_score(Y_test, labels_proba_test[:, 1])
elif label_type == 'multiclass':
    test_roc_auc = roc_auc_score(Y_test, labels_proba_test, multi_class='ovr', average='weighted')
print(f'Test ROC AUC: {test_roc_auc:.3f}')

Check intersection between train/val and test : set()
Test ROC AUC: 0.643


In [9]:
# save auc values and associated embds
auc_values = {'model': embds_dir, 'Cross-val AUC': cross_val_roc_auc, 'test_AUC': test_roc_auc}

In [10]:
list_auc_values.append(auc_values)

In [11]:
list_auc_values

[{'model': 'linear_eval_FIP',
  'Cross-val AUC': np.float64(0.5876765941198931),
  'test_AUC': np.float64(0.6433172302737521)}]

In [12]:
# add new lines to csv
import csv
output_file = '/neurospin/dico/jlaval/Output/V1_FINAL_06-2025/embeddings_classification_results.csv'
with open(output_file, 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=list_auc_values[0].keys())
    if f.tell() == 0:  # Check if file is empty
        writer.writeheader()  # Write header only if file is empty
    writer.writerows(list_auc_values)

# Regression

SC-sylv left

In [13]:
# parameters
## Champollion
embds_dir = '/neurospin/dico/jlaval/Output/V1_FINAL_06-2025/SC-sylv_left/name07-58-00_111/hcp_random_embeddings/train_val_embeddings.csv'
## B-VAE
embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/SC-sylv_left/32dims/unformated_embeddings.csv' # 32 dims is best
embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/SC-sylv_left/75dims/unformated_embeddings.csv'
embds_dir = '/neurospin/dico/jlaval/Output/B-VAE_40k/SC-sylv_left/256dims/unformated_embeddings.csv'
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels_SC-sylv_left.csv'
label_list = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/Isomap/splits/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/Isomap/splits/test_split.csv'
subject_name = 'Subject'

In [18]:
mode = 'linear'

## Linear evaluation

In [19]:
embds_dir = 'linear_eval_isomap_central' ## just for saving results
labels_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels_SC-sylv_left.csv'
label_list = [f'Isomap_central_left_dim{k}' for k in range(1,7)]
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/Isomap/splits/train_val_split_'
test_subs_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/Isomap/splits/test_split.csv'
subject_name = 'Subject'
label_type = 'continuous'

# load the images
skeletons = np.load('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.C.-sylv./mask/Lskeleton.npy')
skel_names = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.C.-sylv./mask/Lskeleton_subject.csv')
# flatten the skeletons
skeletons = skeletons.reshape(skeletons.shape[0], -1)
# binarize the skeletons
skeletons[skeletons > 0] = 1

In [22]:
cross_val_r2_list = []
test_r2_list = []

for label in label_list:

    ## embeddings case
    # load
    # embds = pd.read_csv(embds_dir)
    # embds.columns = ['ID'] + [f'dim{i}' for i in range(embds.shape[1]-1)]

    ## skeletons case
    # create a DataFrame
    embds = pd.DataFrame(skeletons, columns=[f'dim{i}' for i in range(skeletons.shape[1])])
    # add ID column
    embds['ID'] = skel_names['Subject']

    # remove duplicates
    embds = embds.drop_duplicates(subset=['ID'])
    labels = pd.read_csv(labels_dir)
    # restrict embds to Subjects with labels
    embds = embds[embds['ID'].isin(labels[subject_name])].reset_index(drop=True)
    # same for labels
    labels = labels[labels[subject_name].isin(embds['ID'])].reset_index(drop=True)


    # align labels and embds on 'ID'
    labels = labels.merge(embds[['ID']], left_on=subject_name, right_on='ID', how='right')
    # order all by ID
    embds = embds.sort_values(by='ID').reset_index(drop=True)
    labels = labels.sort_values(by='ID').reset_index(drop=True)
    subjects = embds['ID']

    # define X, Y and subjects
    X = embds.drop(columns=['ID'])
    # Standardize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    Y = labels[label]

    subs_embeddings = pd.DataFrame({'ID': subjects, 'X': list(X), 'Y': Y})

    root_dir = '/'.join(splits_basedir.split('/')[:-1])
    basedir = splits_basedir.split('/')[-1]
    splits_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir) if f.startswith(basedir) and '.csv' in f]
    splits_subs = [pd.read_csv(file, header=None) for file in splits_dirs]
    labels = np.concatenate([[i] * len(K) for i, K in enumerate(splits_subs)])
    splits_subs_and_labels = pd.concat(splits_subs)
    splits_subs_and_labels.columns=['ID']
    splits_subs_and_labels['labels'] = labels
    df = subs_embeddings.merge(splits_subs_and_labels, on='ID')
    groups, X_train_val, Y_train_val = df['labels'], np.vstack(df['X'].values), df['Y']
    logo = LeaveOneGroupOut()
    cv = [*(logo.split(X_train_val, Y_train_val, groups=groups))]

    if mode == 'linear':
        parameters={'l1_ratio': np.linspace(0,1,11), 'alpha': [10**k for k in range(-3,4)]} 
        model = ElasticNet(max_iter=10000)
        clf = GridSearchCV(model, parameters, cv=cv, scoring='r2', refit=True, n_jobs=-1)
    elif mode == 'knn':
        parameters = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15], 'leaf_size': [1, 5, 10, 20, 30], 'weights': ['uniform', 'distance'], 'metric': ['minkowski', 'manhattan', 'cosine']}
        model = KNeighborsRegressor(n_jobs=-1)
        clf = GridSearchCV(model, parameters, cv=cv, scoring='r2', refit=True, n_jobs=-1)

    # fit cross-validation
    clf.fit(X_train_val,Y_train_val)
    print(f'best params : {clf.best_params_}')
    print(f'best score : {clf.best_score_}')
    md = clf.best_estimator_
    # compute r2 on cross-validation
    cross_val_r2 = cross_val_score(md, X_train_val, Y_train_val, cv=cv, scoring='r2')
    print(f'Cross-val R2: {cross_val_r2.mean():.3f}')

    # use parameters for test
    test_subjects = pd.read_csv(test_subs_dir, header=None)
    test_subjects.columns = ['ID']
    subs_embeddings_test = subs_embeddings.merge(test_subjects, on='ID')
    # check that there is no intersection between train val and test subjects
    print(f"Check intersection between train/val and test : {set(splits_subs_and_labels['ID']).intersection(set(subs_embeddings_test['ID']))}")
    X_test, Y_test = np.vstack(subs_embeddings_test['X'].to_numpy()), subs_embeddings_test['Y']
    # take best parameters obtained in cross-validation and fit on train+val ## NO NEED TO DO THIS, ALREADY DONE WITH REFIT=TRUE
    # if mode == 'linear':
    #     print('Using linear probing')
    #     md = ElasticNet(max_iter=10000, **clf.best_params_)  # NB : can directly md.fit ?
    # elif mode == 'knn':
    #     print('Using KNN')
    #     md = KNeighborsRegressor(n_jobs=-1, **clf.best_params_)
    # md.fit(X_train_val,Y_train_val)
    # compute r2 on test
    test_r2 = md.score(X_test, Y_test)
    print(f'Test R2: {test_r2:.3f}')

    cross_val_r2_list.append(cross_val_r2)
    test_r2_list.append(test_r2)

/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.093e+00, tolerance: 4.547e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.082e+00, tolerance: 4.482e-01 Linear regression models with null weight for the l1 r

best params : {'alpha': 0.1, 'l1_ratio': np.float64(0.0)}
best score : 0.2488597108821363


/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.051e+01, tolerance: 4.702e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.031e+01, tolerance: 4.547e-01 Linear regression models with null weight for the l1 r

Cross-val R2: 0.249
Check intersection between train/val and test : set()
Test R2: 0.385


/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.555e-01, tolerance: 3.825e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.486e-01, tolerance: 3.665e-01 Linear regression models with null weight for the l1 r

...

Cross-val R2: 0.101
Check intersection between train/val and test : set()
Test R2: 0.190


/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.392e-01, tolerance: 1.191e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.355e-01, tolerance: 1.226e-01 Linear regression models with null weight for the l1 r

best params : {'alpha': 10, 'l1_ratio': np.float64(0.0)}
best score : 0.07399568411231239


/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.107e+02, tolerance: 1.263e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.055e+02, tolerance: 1.209e-01 Linear regression models with null weight for the l1 r

Cross-val R2: 0.074
Check intersection between train/val and test : set()
Test R2: 0.107


In [23]:
# average r2
cross_val_r2_mean = np.mean(cross_val_r2_list)
test_r2_mean = np.mean(test_r2_list)
print(f'Average Cross-val R2: {cross_val_r2_mean:.3f}')
print(f'Average Test R2: {test_r2_mean:.3f}')

Average Cross-val R2: 0.172
Average Test R2: 0.230


In [24]:
# save r2 values and associated embds dir
r2_values = {'model': embds_dir, 'Cross-val R2': cross_val_r2_mean, 'test_R2': test_r2_mean}

In [25]:
# add new lines to csv
output_file = '/neurospin/dico/jlaval/Output/V1_FINAL_06-2025/embeddings_regression_results.csv'
with open(output_file, 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=r2_values.keys())
    if f.tell() == 0:  # Check if file is empty
        writer.writeheader()  # Write header only if file is empty
    writer.writerow(r2_values)